In [86]:
# Run in python console
import nltk; nltk.download('stopwords')

# Run in terminal or command prompt
!python3 -m spacy download en

[nltk_data] Downloading package stopwords to /home/nidhal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2022-06-16 01:56:02.212195: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-16 01:56:02.212262: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 12.8 MB 345 kB/s            
You should consider upgrading via the '/home/nidhal/miniconda3/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [87]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [88]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [89]:
# Import Dataset
df = pd.read_csv('./question_responce.csv')
print(df.question.unique())
df.head()

['@AmericanAir Erica on the lax team is amazing give her a raise ty'
 '@AmericanAir Could you have someone on your lax team available to guide me to my gate ASAP'
 'Ben Tennyson and an American Airlines pilot. 🎃 #trunkortreat #halloween #2017 #diycostume #parenting @americanair … https://t.co/f1nNHQ0iLa https://t.co/lDViDkRdB1'
 ...
 '@AmericanAir Really annoyed been over a month since my damaged bag claim never heard back! done as told...'
 '@AmericanAir terrible service wait ages trying to call that number almost two months gone no response'
 '@AmericanAir charges their patrons to change their flights every time an airport closes #stcroix']


,Unnamed: 0,responce,question
0,603,@115904 We'll be sure to pass along your kind ...,@AmericanAir Erica on the lax team is amazing ...
1,605,@115904 Our apologies for the delay in respond...,@AmericanAir Could you have someone on your la...
2,608,"@115905 Aww, that's definitely a future pilot ...",Ben Tennyson and an American Airlines pilot. 🎃...
3,612,@115906 We're sorry for your frustration.,"@AmericanAir Right, but I earned those. I also..."
4,618,@115909 We're glad you got to kick back and en...,"Thank you, @AmericanAir for playing #ThisIsUs ..."


In [90]:
# Convert to list
data_responce = df.responce.values.tolist()
data_question = df.question.values.tolist()

# Remove Emails
data_responce = [re.sub('\S*@\S*\s?', '', sent) for sent in data_responce]
data_question = [re.sub('\S*@\S*\s?', '', sent) for sent in data_question]

# Remove new line characters
data_responce = [re.sub('\s+', ' ', sent) for sent in data_responce]
data_question = [re.sub('\s+', ' ', sent) for sent in data_question]

# Remove distracting single quotes
data_responce = [re.sub("\'", "", sent) for sent in data_responce]
data_question = [re.sub("\'", "", sent) for sent in data_question]

pprint(data_question[:1])

['Erica on the lax team is amazing give her a raise ty']


In [91]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data_question))

print(data_words[:1])

[['erica', 'on', 'the', 'lax', 'team', 'is', 'amazing', 'give', 'her', 'raise', 'ty']]


In [92]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['erica', 'on', 'the', 'lax', 'team', 'is', 'amazing', 'give', 'her', 'raise', 'ty']


In [93]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [94]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['team', 'amazing', 'give', 'raise', 'ty']]


In [95]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [96]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [97]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.106*"know" + 0.091*"tell" + 0.074*"issue" + 0.069*"leave" + '
  '0.056*"minute" + 0.044*"happen" + 0.041*"board" + 0.040*"fix" + 0.032*"due" '
  '+ 0.023*"arrive"'),
 (1,
  '0.207*"want" + 0.130*"customer" + 0.065*"answer" + 0.051*"expect" + '
  '0.002*"resolution" + 0.000*"refund" + 0.000*"stop" + 0.000*"provide" + '
  '0.000*"deal" + 0.000*"beconsistent"'),
 (2,
  '0.247*"bag" + 0.198*"check" + 0.090*"call" + 0.044*"number" + 0.035*"team" '
  '+ 0.034*"phone" + 0.024*"enough" + 0.015*"status" + 0.013*"asap" + '
  '0.012*"row"'),
 (3,
  '0.119*"book" + 0.085*"keep" + 0.074*"free" + 0.057*"price" + 0.054*"wifi" + '
  '0.048*"next" + 0.045*"least" + 0.030*"available" + 0.028*"fare" + '
  '0.028*"finally"'),
 (4,
  '0.097*"early" + 0.079*"look" + 0.068*"boarding" + 0.055*"awesome" + '
  '0.046*"purchase" + 0.042*"food" + 0.041*"end" + 0.033*"card" + '
  '0.033*"platinum" + 0.030*"aadvantage"'),
 (5,
  '0.139*"send" + 0.084*"month" + 0.073*"receive" + 0.054*"email" + '
  '0.046*

In [98]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -15.076927643005043

Coherence Score:  0.45330333415745727


In [99]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17    -0.370403 -0.244595       1        1  16.964194
16    -0.179869  0.306359       2        1  11.485420
8     -0.153175  0.156443       3        1  11.212032
9     -0.017673  0.003961       4        1   5.840424
0      0.027864 -0.012376       5        1   5.558597
15    -0.000180 -0.003243       6        1   5.537870
19     0.011023 -0.006870       7        1   5.500800
2      0.016336 -0.007827       8        1   4.359627
13     0.029481 -0.011151       9        1   3.971096
18     0.051919 -0.016743      10        1   3.773367
14     0.039412 -0.013507      11        1   3.747851
4      0.053946 -0.016445      12        1   3.474710
11     0.047383 -0.015305      13        1   3.447368
12     0.050244 -0.015628      14        1   3.308094
10     0.051313 -0.015729      15        1   3.121220
3      0.059314 -0.017094      16        1   2.703165
7      0.061495 -0.017372      17        1   2.681231
5      0.064085 -0.017793      18        1   1.665471
1      0.073549 -0.018222      19        1   1.529022
6      0.083937 -0.016864      20        1   0.118440, topic_info=          Term        Freq       Total Category  logprob  loglift
27      flight  908.000000  908.000000  Default  30.0000  30.0000
108        get  420.000000  420.000000  Default  29.0000  29.0000
31       thank  294.000000  294.000000  Default  28.0000  28.0000
154       time  289.000000  289.000000  Default  27.0000  27.0000
103        fly  186.000000  186.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
25   attendant    0.006229   41.428679  Topic20  -8.0107  -2.0641
26        back    0.006229   74.250322  Topic20  -8.0107  -2.6475
27      flight    0.006229  908.624449  Topic20  -8.0107  -5.1520
28       great    0.006229   94.157671  Topic20  -8.0107  -2.8851
29        home    0.006229   82.355987  Topic20  -8.0107  -2.7511

[767 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
421       7  0.985328          aa
206      12  0.904266  aadvantage
72       17  0.964853        able
398       6  0.542070      accord
79       17  0.700721    addition
...     ...       ...         ...
409       1  0.924073      window
42        5  0.845676   wonderful
329       6  0.983699        work
456      10  0.880961       world
118       6  0.970091        year

[341 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 17, 9, 10, 1, 16, 20, 3, 14, 19, 15, 5, 12, 13, 11, 4, 8, 6, 2, 7])

In [100]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [101]:
!pip install gensim==3.8.3

You should consider upgrading via the '/home/nidhal/miniconda3/bin/python -m pip install --upgrade pip' command.


In [102]:
#import os
#os.environ['MALLET_HOME'] = '../mallet-2.0.8'
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = '../mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

Mallet LDA: 20 topics, 5 topic bits, 11111 topic mask
Data loaded.
max tokens: 25
total tokens: 12822
<10> LL/token: -9,20294
<20> LL/token: -8,9252
<30> LL/token: -8,7667
<40> LL/token: -8,68695

0	2,5	agent customer pay plan nice minute answer board refund book pass arrive fare disappoint phone end bring computer fly bad 
1	2,5	ticket free day check extra aadvantage pay aa seat team finally cost clt charge expensive lose place boarding drink counter 
2	2,5	travel home today holiday hope hold pilot time make awful claim flight switch entire ill break bit sweet website case 
3	2,5	early airport good wait attendant flight minute leave gate online boarding process show pretty plenty deliver crazy wedding dog end 
4	2,5	cancel flight wait delay refund morning ago pay class company chance provide ridiculous part point offer long policy status system 
5	2,5	make https flight update fee love expect first_class departure suck terminal rude unacceptable purchase mess enjoy delay malfunction ca

<160> LL/token: -8,41368
<170> LL/token: -8,41672
<180> LL/token: -8,41293
<190> LL/token: -8,39728

0	2,5	agent customer minute boarding board start answer platinum line plan end computer arrive multiple place personal desk mistake important complimentary 
1	2,5	pay ticket book charge free extra price follow cost expensive destination tweet question phx drive overhead half request premium warn 
2	2,5	travel holiday home hope happy ridiculous understand job staff system claim friend date life son entire ill hold drink wrong 
3	2,5	good day fly wait happen buy awesome flight bring award notice favorite row shout address airport assist hear reply big 
4	2,5	flight cancel refund full week offer terrible travel passenger sell rebook lack broken window people american worry huge visit announce 
5	2,5	make early fee update expect work close love sit first_class wifi aadvantage departure tonight fit mess policy sense program app 
6	2,5	seat amp find lose fly room pass member cool include turn

<310> LL/token: -8,34588
<320> LL/token: -8,33839
<330> LL/token: -8,3666
<340> LL/token: -8,378

0	2,5	agent customer minute boarding board start answer hold unacceptable platinum arrive treat date super question walk include economy desk group 
1	2,5	pay ticket book charge refund tomorrow free chance nice follow extra cabin cost guy price expensive car meet airplane frustrating 
2	2,5	travel home experience holiday plan hope understand reason cool info friend figure fix kid kudo screen huge crazy hate completely 
3	2,5	fly good day buy voucher guess dm fix bring wife company night depart mechanical case wait excellent fee state credit 
4	2,5	flight cancel day refund pass change reservation week real speak airport sell mess account loyal guarantee booking assign pick terrible 
5	2,5	today early sit update option departure first_class ago wifi love claim flight compensation air put close empty talk expect nice 
6	2,5	seat amp find lose fly card member purchase life staff club differenc

<460> LL/token: -8,36367
<470> LL/token: -8,35544
<480> LL/token: -8,33007
<490> LL/token: -8,31669

0	2,5	agent minute fix board start boarding hold platinum put arrive group response question empty process answer place damage clt sweet 
1	2,5	pay ticket book customer charge refund price lounge cost expensive extra phx excellent mail double literally speak round earn ac 
2	2,5	travel experience holiday plan understand hope person home option child completely life disappoint deny disappointed idea assist resolve aa money 
3	2,5	fly good trip voucher tomorrow buy week month open bring make wife figure lot carrier handle line overhead son wrong 
4	2,5	flight cancel day food switch ground notice advance sell aircraft guarantee super half american_airline strand bit upset scheduling rebooke hotel 
5	2,5	work early nice love back expect close first_class end aadvantage wifi spend cancellation option reason air extra account line ve 
6	2,5	seat lose amp find problem guess card member fare sc

<610> LL/token: -8,34074
<620> LL/token: -8,32252
<630> LL/token: -8,33959
<640> LL/token: -8,34213

0	2,5	agent minute boarding board start hold answer departure arrive unacceptable hotel part question computer vacation announce people meet window stay 
1	2,5	pay ticket book customer charge refund extra online fare cost price award expensive ahead carryon regard select assure reschedule shame 
2	2,5	travel experience holiday mile hope fee happy person disappoint ill life totally completely special weekend domestic complete huge power airport 
3	2,5	good trip buy tomorrow week put team return line lot process locator big access captain sweet app refuse air bag 
4	2,5	flight cancel family stop tonight suck system glitch strand end car advance arrangement entire guarantee joke half scheduling record unable 
5	2,5	work back early nice love option expect feel first_class aadvantage small information spend head club cabin stick counter schedule desk 
6	2,5	seat amp lose find problem card me

<760> LL/token: -8,34786
<770> LL/token: -8,32467
<780> LL/token: -8,30379
<790> LL/token: -8,30019

0	2,5	wait agent minute board answer ridiculous start phone put hold question long row computer finally yesterday resolve poor mistake forward 
1	2,5	pay ticket book charge refund extra online fare price date fail cost expensive record locator pick code smooth earn unable 
2	2,5	travel experience mile expect hope holiday understand unacceptable person fee system disappoint affect compensation real life amp customer_service boarding program 
3	2,5	good trip customer people return response money work arrive figure city place son buy american personal idea apparently sky curious 
4	2,5	flight cancel leave option family hotel drive account car middle ruin original result ground stay customer preferred cruise credit_card apply 
5	2,5	early work nice love fix feel first_class long small spend end information app awful club screw enjoy power outlet specific 
6	2,5	seat find lose card member ch

<910> LL/token: -8,29296
<920> LL/token: -8,27814
<930> LL/token: -8,2946
<940> LL/token: -8,30044

0	2,5	wait agent minute morning board hold phone min put arrive departure affect wife ill finally ridiculous computer vacation sense poor 
1	2,5	pay ticket book charge refund fee extra online fare cost price expensive record multiple counter locator folk message cover unable 
2	2,5	travel experience mile hope boarding start problem pass small group holiday turn kid life award program rule pretty space forward 
3	2,5	customer good trip people return buy understand big include policy city figure lot strand money son hr idea concerned link 
4	2,5	flight cancel option leave screw international part list assist ahead worry door disappointed catch overnight lol operate scheduling apply hear 
5	2,5	early work nice happen update feel first_class leave information spend end close lounge disappoint date app tweet dog club sad 
6	2,5	seat find lose free crew member purchase card aadvantage person p

In [103]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

[(8,
  [('guy', 0.0732484076433121),
   ('call', 0.07165605095541401),
   ('home', 0.06847133757961783),
   ('send', 0.05573248407643312),
   ('back', 0.03821656050955414),
   ('answer', 0.03184713375796178),
   ('receive', 0.03184713375796178),
   ('info', 0.030254777070063694),
   ('email', 0.025477707006369428),
   ('contact', 0.025477707006369428)]),
 (18,
  [('time', 0.2064),
   ('give', 0.0768),
   ('ve', 0.0496),
   ('happy', 0.04),
   ('today', 0.0368),
   ('month', 0.0304),
   ('number', 0.0304),
   ('status', 0.0224),
   ('hear', 0.0176),
   ('wife', 0.0176)]),
 (1,
  [('pay', 0.13037037037037036),
   ('ticket', 0.10518518518518519),
   ('book', 0.0962962962962963),
   ('refund', 0.0637037037037037),
   ('charge', 0.05037037037037037),
   ('fee', 0.034074074074074076),
   ('buy', 0.03259259259259259),
   ('expect', 0.02666666666666667),
   ('extra', 0.02666666666666667),
   ('online', 0.022222222222222223)]),
 (10,
  [('hour', 0.17346938775510204),
   ('delay', 0.170553935860

In [104]:
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


Coherence Score:  0.5526024309143509


In [105]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

Mallet LDA: 2 topics, 1 topic bits, 1 topic mask
Data loaded.
max tokens: 25
total tokens: 12822
<10> LL/token: -8,0933
<20> LL/token: -7,98013
<30> LL/token: -7,94613
<40> LL/token: -7,90062

0	25	flight seat plane gate time ticket delay pay travel aa book bag wait bad agent charge call https passenger airport 
1	25	flight make fly https check day hour bag change give cancel service home time good delay pilot miss guy work 

<50> LL/token: -7,87958
<60> LL/token: -7,86919
<70> LL/token: -7,85436
<80> LL/token: -7,84739
<90> LL/token: -7,83309

0	25	flight seat gate plane pay time travel hour ticket book delay fly wait today bad agent back charge call issue 
1	25	flight https bag make fly check day great aa change give delay airline time service miss home customer pilot cancel 

<100> LL/token: -7,82112
<110> LL/token: -7,82103
<120> LL/token: -7,82142
<130> LL/token: -7,81493
<140> LL/token: -7,80811

0	25	flight seat gate delay plane fly pay travel ticket today book time wait bad age

<60> LL/token: -8,24724
<70> LL/token: -8,20284
<80> LL/token: -8,17786
<90> LL/token: -8,16798

0	6,25	seat flight ticket https boarding crew holiday give people family due bad mile board receive number card put buy stop 
1	6,25	make flight day cancel guy https book home business long option food tomorrow close contact small information reservation reason confirm 
2	6,25	flight pay wait customer passenger early attendant hour voucher happy fix refund amazing rude member extra offer ve aadvantage hold 
3	6,25	travel flight service miss agent gate call today fly sit issue amp find bad nice late connection expect job chance 
4	6,25	flight time hour pilot leave great problem book give plan hope team answer phone issue awesome pass hear company free 
5	6,25	upgrade flight back charge time good trip work great experience airport home class full week show purchase end schedule line 
6	6,25	delay fly aa flight change airline year lose hour land response love thing happen gate understand depar

<410> LL/token: -8,01864
<420> LL/token: -8,03534
<430> LL/token: -8,0172
<440> LL/token: -8,01064

0	6,25	seat ticket good give people amp leave holiday due hope happen option fix buy card week stop put small return 
1	6,25	https make day book bad airline back home board tomorrow expect dm contact min helpful unacceptable confirm provide terrible cancellation 
2	6,25	pay wait flight customer passenger attendant voucher morning happy staff business start rude close amazing extra pass aadvantage first_class line 
3	6,25	delay hour today miss service call airport boarding refund free find nice response late connection family love awesome thanksgiving show 
4	6,25	flight time travel issue pilot minute mile plan problem fee answer phone month trip ago online tonight part schedule vacation 
5	6,25	great upgrade charge guy work year experience time trip long class receive info feel hold platinum member hear money chance 
6	6,25	flight fly aa cancel change lose ve food thing number understand

<760> LL/token: -8,01421
<770> LL/token: -8,00884
<780> LL/token: -8,01254
<790> LL/token: -7,99667

0	6,25	seat ticket give back aa people ve option connection fix platinum put show ago hear money purchase small wifi online 
1	6,25	https make book airline bad home refund day find team full job miss information helpful confirm point affect speak love 
2	6,25	flight pay good wait customer passenger amp attendant problem rude thing morning dm tomorrow voucher close staff first_class contact line 
3	6,25	delay hour plane today service call airport miss crew nice boarding family late amazing stop connect love thanksgiving maintenance big 
4	6,25	time travel fly issue minute day experience plan due response leave board business start phone answer month holiday guess end 
5	6,25	fly great upgrade guy work charge trip year mile update return hope class fee expect info long pass ridiculous aadvantage 
6	6,25	flight cancel change pilot aa lose happy food number understand offer schedule company

<60> LL/token: -8,40076
<70> LL/token: -8,35636
<80> LL/token: -8,33747
<90> LL/token: -8,30979

0	3,57143	time ticket flight work travel response platinum aadvantage first_class person online date night fly deal screw empty award special lol 
1	3,57143	flight change passenger sit https luggage airport option aa class fee guess late miss open phone cool claim member dog 
2	3,57143	cancel aa attendant due year voucher close ridiculous email flight wifi company cancellation treat price affect website lot yesterday international 
3	3,57143	flight customer charge find carry update plan thing board answer fix week put expect ago room reservation food friend add 
4	3,57143	https hour plane flight guy crew lose problem min land amazing information reason year home include tonight super leg screen 
5	3,57143	flight delay pay back trip leave holiday full buy chance offer ruin depart son rule beautiful hr window wedding landing 
6	3,57143	issue pilot fly mile good love happy morning receive move

...

<510> LL/token: -8,70062
<520> LL/token: -8,68701
<530> LL/token: -8,69909
<540> LL/token: -8,73504

0	1,31579	bag check carry bring force bad program age pleasant awake measure op man appal fair destroy sjc difference call bootleg 
1	1,31579	day charge lose find claim club dollar concerned mistake process write voucher destination avoid small total expedite big pm suitcase 
2	1,31579	flight crew update guess stop free card malfunction ground aa meet fix true drink direct hurt review imaginary reward cust 
3	1,31579	cancel refund book wifi expect date advance yesterday screw worry visit smooth fuck stuff guarantee bo week issue surprise ruin 
4	1,31579	travel holiday food voucher love full real service kid thanksgiving staff security cheap ease enjoy ill presentation stick limited checkin 
5	1,31579	back guy send info board unacceptable sad damage explain deplane email free terminal equipment huge form resolve demand reasonable live 
6	1,31579	ticket problem tomorrow morning plan first

<610> LL/token: -8,71293
<620> LL/token: -8,69656
<630> LL/token: -8,68772
<640> LL/token: -8,68692

0	1,31579	bag check carry force plenty load print tv pick svc major sizer delta straight break cabin transatlantic ramp cleaning zxpypetq 
1	1,31579	day find lose charge cancel process carrier dollar talk airline bring ease flyer short expedite review swear prior son specific 
2	1,31579	flight crew stop provide ground worry confirm big presentation notify jet mother locate super steal wth deplane lady dumb pathetic 
3	1,31579	book refund cancel full wifi ruin small part assist response charlotte info open game pick difference snack service flt compensate 
4	1,31579	travel holiday voucher food friend promise avoid meal gift wedding age top read appreciated prior menu limited standard normal merger 
5	1,31579	back send free guy helpful damage hotel address open tweet fill equipment link form ord advice plastic exceptional set absolute 
6	1,31579	ticket morning problem attendant tomorrow p

<710> LL/token: -8,68187
<720> LL/token: -8,68122
<730> LL/token: -8,68838
<740> LL/token: -8,71224

0	1,31579	bag check carry fee force start plenty complimentary airport checkin middle double passanger group counter teach kinda make americanairbad impress 
1	1,31579	day find lose add process announce american_airline employee talk character standard reimburse future past long wonderful price expectation enjoy stick 
2	1,31579	flight feel hear confirm row show number guarantee waste end recent rebook guess site cancel meeting offer comfy news incident 
3	1,31579	book cancel refund holiday full line case bring credit weekend nice extremely reschedule offer worry direction huge stop provide glad 
4	1,31579	travel voucher food land month meal provide head plane son special mail holiday mention wrong bring support redeem brand identify 
5	1,31579	free guy back send information wifi hold price totally rep piece offer fill equipment demand schedule frustrating booking wound house 
6	1,31579

<810> LL/token: -8,74125
<820> LL/token: -8,69014
<830> LL/token: -8,70392
<840> LL/token: -8,73218

0	1,31579	bag check carry fit small force free daughter human smile space military news safe black highly miserable upg caste impress 
1	1,31579	day lose find start member process announce special mail lady schedule american_airline worth apparently turn lax unaccompanied relation dinner enjoy 
2	1,31579	flight feel passenger happy lot dca shout phx water customerservice premium sky advance age affected quick safety high update fine 
3	1,31579	book refund cancel holiday full understand happen family update guarantee worry meet recognize main customer_service social_media swear explain lounge booking 
4	1,31579	travel food voucher international true move fail walk rep catering thankful night family hand search eye airline inconvenience hub infant 
5	1,31579	back guy send response passenger damage wrong money regard game twitter equipment head cust fund question email insist studio notifi

<910> LL/token: -8,70719
<920> LL/token: -8,69113
<930> LL/token: -8,67286
<940> LL/token: -8,67164

0	1,31579	bag check carry fit force guess fault group free drop hub confirm mind ginger reduce fair credit_card flying member pleasant 
1	1,31579	day lose find start club process head ruin speak ahead charge life rip fee guy comfortable delaaye super carry mile 
2	1,31579	flight happy home morning yesterday update water talk departure curious apparently ac cover lga guarantee feel pro dinner celebrate highly 
3	1,31579	book cancel refund understand aa end prefer mail inform pain amount error lame tv holiday thought future age flat worry 
4	1,31579	travel voucher family food feel passenger son credit completely walk arrangement hear check security real aadvantage happen appreciated forward rebooke 
5	1,31579	guy back send response hard damage totally departure specific visit pull equipment garbage water form shoutout traffic follow fill wth 
6	1,31579	ticket buy problem price tomorrow co

In [106]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(0,
  '0.077*"call" + 0.065*"year" + 0.057*"guy" + 0.050*"nice" + 0.045*"update" + '
  '0.045*"amp" + 0.033*"ve" + 0.032*"week" + 0.030*"receive" + 0.023*"status"'),
 (1,
  '0.113*"cancel" + 0.086*"airline" + 0.066*"trip" + 0.035*"expect" + '
  '0.033*"return" + 0.030*"buy" + 0.024*"chance" + 0.021*"full" + '
  '0.020*"price" + 0.018*"include"'),
 (2,
  '0.208*"seat" + 0.134*"pay" + 0.034*"move" + 0.026*"extra" + '
  '0.025*"purchase" + 0.022*"information" + 0.022*"room" + 0.017*"group" + '
  '0.017*"screw" + 0.017*"hotel"'),
 (3,
  '0.165*"plane" + 0.091*"fly" + 0.067*"airport" + 0.049*"sit" + 0.034*"start" '
  '+ 0.021*"spend" + 0.017*"screen" + 0.017*"ill" + 0.017*"deal" + '
  '0.015*"kid"'),
 (4,
  '0.190*"delay" + 0.184*"hour" + 0.082*"miss" + 0.038*"late" + '
  '0.035*"connection" + 0.020*"suck" + 0.020*"flight" + 0.019*"policy" + '
  '0.016*"refund" + 0.016*"water"'),
 (5,
  '0.081*"aa" + 0.081*"give" + 0.075*"pilot" + 0.040*"plan" + 0.037*"option" + '
  '0.035*"food" + 0.032*"

In [112]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_question, responses=data_responce):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    contents_response = pd.Series(responses)
    sent_topics_df = pd.concat([sent_topics_df, contents, contents_response], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data_question, responses=data_responce)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Question', 'Response']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Question,Response
0,0,0.0,0.0657,"call, year, guy, nice, update, amp, ve, week, ...",Erica on the lax team is amazing give her a ra...,Well be sure to pass along your kind words! #A...
1,1,14.0,0.0939,"gate, wait, agent, minute, leave, board, phone...",Could you have someone on your lax team availa...,Our apologies for the delay in responding to y...
2,2,5.0,0.1141,"aa, give, pilot, plan, option, food, answer, s...",Ben Tennyson and an American Airlines pilot. 🎃...,"Aww, thats definitely a future pilot in the ma..."
3,3,15.0,0.0685,"flight, passenger, crew, love, time, provide, ...","Right, but I earned those. I also shouldn’t ha...",Were sorry for your frustration.
4,4,11.0,0.0692,"flight, great, staff, terrible, destination, e...","Thank you, for playing #ThisIsUs and for havin...",Were glad you got to kick back and enjoy a sho...
5,5,16.0,0.0876,"good, upgrade, work, people, send, mile, amazi...",wifi makes Amtraks wifi look pro. At least the...,We never want your experience to be anything l...
6,6,3.0,0.0660,"plane, fly, airport, sit, start, spend, screen...",Wonderful club! American Airlines Admirals Clu...,"Thanks for the shout-out and for stopping by, ..."
7,7,12.0,0.0887,"make, change, customer, hope, thing, fee, dm, ...","already did...changed browsers, did all my tec...",Theyll be more than happy to walk you through ...
8,8,1.0,0.0636,"cancel, airline, trip, expect, return, buy, ch...",........still....on....hold.....they made the ...,Were sorry for the long wait. The next availab...
9,9,19.0,0.0789,"ticket, book, charge, refund, number, ridiculo...",well now i am told the ticket cost is an addit...,Fares sell in real time. Were sorry we werent ...


In [113]:
df_dominant_topic.to_csv("dominant_topic.csv", sep=";")